In [ ]:
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5awssxz7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5awssxz7
  Resolved https://github.com/huggingface/transformers.git to commit 0abf5e8eaec361350888517f0d98501b2c5b2559
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip uninstall pyarrow requests -y



Found existing installation: pyarrow 14.0.1
Uninstalling pyarrow-14.0.1:
  Successfully uninstalled pyarrow-14.0.1
Found existing installation: requests 2.31.0
Uninstalling requests-2.31.0:
  Successfully uninstalled requests-2.31.0


In [ ]:
!pip install wheel




In [ ]:
!pip install transformers torchaudio


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio


In [ ]:
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

audio_file = next(iter(uploaded.keys()))
waveform, sample_rate = torchaudio.load(audio_file)


Saving sample.wav to sample.wav


In [ ]:
resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)


In [ ]:
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)


In [ ]:
inputs = processor(waveform.squeeze(), return_tensors="pt", sampling_rate=16000)


In [ ]:
with torch.no_grad():
    predicted_ids = model.generate(inputs.input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(transcription)


 Так, на казахском отправлю, на узбекском нету. На узбекском, кажется, не отправляли. Сейчас надо посмотреть.


In [ ]:
!pip install transformers torchaudio soundfile


In [ ]:
!pip install pydub

In [ ]:
!pip install transformers torchaudio pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from google.colab import files
import subprocess

# Upload the audio file
uploaded = files.upload()
audio_file = next(iter(uploaded.keys()))

# Convert the .ogg file to .wav using ffmpeg
wav_file = audio_file.replace('.ogg', '.wav')
subprocess.run(['ffmpeg', '-i', audio_file, wav_file])


Saving kazakh1.ogg to kazakh1 (3).ogg


CompletedProcess(args=['ffmpeg', '-i', 'kazakh1 (3).ogg', 'kazakh1 (3).wav'], returncode=0)

In [ ]:
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Load the .wav audio file using torchaudio
waveform, sample_rate = torchaudio.load(wav_file)

# Resample the audio to 16000 Hz
resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)

# Ensure the waveform is mono
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# Load the model and processor
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Prepare the input for the model
inputs = processor(waveform.squeeze(), return_tensors="pt", sampling_rate=16000)

# Generate the transcription
with torch.no_grad():
    predicted_ids = model.generate(inputs.input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Display the transcription
print(transcription)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
P

 Кайылы кеш ренджмен сәтес мұзаламышын ақсы айтып жатырмаған досып келмейді деп.
